In [3]:
import pandas as pd
import pymongo
from pymongo import MongoClient
import graphlab as gl
import csv
import collections
import datetime
from graphlab.toolkits.model_parameter_search import model_parameter_search
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.wordnet import WordNetLemmatizer
gl.canvas.set_target('ipynb')

In [4]:
client = MongoClient()
kiva = client.kiva
mongo_loans = kiva.loans
mongo_lenders_loans = kiva.lenders_loans

##Feature Engineering
This iteration will consider only text features, i.e. Tfidf of loan description.

In [ ]:
# get lenders_loans
cursor_lenders_loans = mongo_lenders_loans.find({}, {'_id':0}).sort('lender_id', 1).limit(100000)
sample_lenders_loans = pd.DataFrame(list(cursor_lenders_loans))
sample_lenders_loans.dropna(inplace=True)

In [ ]:
# create lender-loan pairs
with open('data/lenders_loans_100000.csv', 'w') as f:
    wr = csv.writer(f, delimiter=',')
    for r in sample_lenders_loans.iterrows():
        for l in r[1]['loan_ids']:
            wr.writerow([r[1]['lender_id'], l])

In [5]:
sf = gl.SFrame.read_csv('data/lenders_loans_100000.csv', header=False, delimiter=',', verbose=False)
sf.rename({'X1':'lender_id', 'X2':'loan_id'})
sf.sort('loan_id')

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/yulizhou/Documents/Projects/zipfian/Kipfian/data/lenders_loans_100000.csv
PROGRESS: Parsing completed. Parsed 1218194 lines in 0.809943 secs.


lender_id,loan_id
NkInteractClub,280242
am3914,280741
ake8008,280758
alistair8793,280758
barbara4752,280766
adrian9794,280766
am3914,280766
aaron83574977,280766
alistair8793,280776
andrew4225,280781


In [6]:
# start getting text
# we only use english version
# start getting features

# get rid of loans in lender_loans but not in loans
feat_loan_id = sf['loan_id'].unique().sort()
ids = list(feat_loan_id)
temp_cursor = mongo_loans.find({'id': {'$in': ids}}, {'id': 1, '_id': 0})
loans_ids_exists = list(temp_cursor)
loans_ids_exists = sorted([l['id'] for l in loans_ids_exists])
diff_ids = set(ids) - set(loans_ids_exists)
sf['loan_id'] = sf['loan_id'].apply(lambda x: x if x not in diff_ids else None)
sf = sf.dropna('loan_id')

In [ ]:
c=mongo_loans.find({'id': {'$in': loans_ids_exists}, 'description.texts.en': {'$exists': True}}, 
                   {'id': 1, 'description.texts.en': 1, '_id':0})\
                    .sort('id', 1)
raw_features = gl.SFrame(list(c))

In [ ]:
# define functions to lemmatize and vectorize text
def lemmatize_descriptions(descriptions):
    lem = WordNetLemmatizer()
    lemmatize = lambda d: " ".join(lem.lemmatize(word) for word in d.split())
    return [lemmatize(desc) for desc in descriptions]

def get_vectorizer(descriptions, num_features=5000):
    vect = TfidfVectorizer(max_features=num_features, stop_words='english')
    return vect.fit(descriptions)

In [ ]:
# create tfidf features
text = lemmatize_descriptions(df['features'].values)
tfidf = pd.DataFrame(get_vectorizer(text).transform(text).toarray())
df.drop(['features'])
df = df.append(tfidf, ignore_index=True)

##Train models

In [ ]:
# convert features into SFrame
df['loan_id'] = df['loan_id'].astype(basestring)
loan_feature = gl.SFrame(df.to_dict(orient='list'))

In [ ]:
# split train test
train, test = gl.recommender.util.random_split_by_user(sf, user_id='lender_id', item_id='loan_id', item_test_proportion=0.2)

In [ ]:
# Train the model
models = []
regs = [0.1, 0.01, 0.001]
num_factors = range(2, 5)
for n in num_factors:
    for r in regs:
        m = gl.recommender.ranking_factorization_recommender.create(train,
                                                                    user_id='lender_id', 
                                                                    item_id='loan_id',
                                                                    item_data=,
                                                                    num_factors=n,
                                                                    regularization=r, 
                                                                    binary_target=True, 
                                                                    verbose=True)
        models.append(m)

In [4]:
df = pd.read_csv('data/loans_lenders_100000.csv', header=None)

In [6]:
df.columns = ['loan_id', 'lender_id']

In [14]:
group_by_loan = df.groupby('loan_id').count().sort('lender_id', ascending=False)
group_by_loan

,lender_id
loan_id,
340316,322
340449,318
345747,315
348881,311
379677,310
344229,309
300405,296
388284,296
389670,292


In [58]:
group_by_loan['lender_id'].mean()

23.432178331946798

In [15]:
group_by_lender = df.groupby('lender_id').count().sort('loan_id', ascending=False)
group_by_lender

,loan_id
lender_id,
zx81,18398
trolltech4460,17699
gooddogg1,14497
jamesclayton9485,7463
stephen3863,4532
joinFITE,4265
barbara5610,4026
emofund,3189
don9212,3095
